In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool, DuckDuckGoSearchResults, WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper
from langchain.document_loaders import WebBaseLoader
from typing import Any, Type
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType

llm = ChatOpenAI(
    temperature=0.1,
)

class SearchToolArgsSchema(BaseModel):
    query: str = Field(description="The query you will search for.Example query: Research about the XZ backdoor")

class WekipediaSearchTool(BaseTool):
    name = "WekipediaSearchTool"
    description = """
    Use this tool to do your research.
    It takes a query as an argument and returns the search results.
    """
    args_schema: Type[
        SearchToolArgsSchema
    ] = SearchToolArgsSchema
    def _run(self, query) -> Any:
        wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
        return wikipedia.run(query)

class DuckDuckGoSearchTool(BaseTool):
    name = "DuckDuckGoSearchTool"
    description = """
    Use this tool to do your research.
    It takes a query as an argument and returns the search results.
    """
    args_schema: Type[
        SearchToolArgsSchema
    ] = SearchToolArgsSchema
    def _run(self, query) -> Any:
        ddg = DuckDuckGoSearchResults()
        return ddg.run(query)

class WebsiteScrapingToolArgsSchema(BaseModel):
    url: str = Field(description="The url you will extract content for.Example query: https://nomadcoders.co/")

class WebsiteScrapingTool(BaseTool):
    name = "WebsiteScrapingTool"
    description = """
    Visit websites and extract their content with this tool.
    It takes a url as an argument and returns the content.
    """
    args_schema: Type[
        WebsiteScrapingToolArgsSchema
    ] = WebsiteScrapingToolArgsSchema
    def _run(self, url) -> Any:
        loader = WebBaseLoader(url)
        data = loader.load()
        return data
    
class TestSaveToolArgsSchema(BaseModel):
    data: str = Field(description="The data you will save the text file for.Example query: the backdoor is a security vulnerability...")

class TextSaveTool(BaseTool):
    name = "TextSaveTool"
    description = """
    Use this tool to save the text to a file.
    It takes a data as an argument.
    """
    args_schema: Type[
        TestSaveToolArgsSchema
    ] = TestSaveToolArgsSchema
    def _run(self, data) -> Any:
        with open("What_is_XZ_backdoor.txt", "w") as f:
            f.write(data)
        return "Text saved to What_is_XZ_backdoor.txt"

agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[WekipediaSearchTool(),
           DuckDuckGoSearchTool(),
           WebsiteScrapingTool(),
           TextSaveTool()],
)

prompt = """
    Search for query. Use the any SearchTool.
    You find a website address, visit the website, extract the content.
    Save your findings to a text file.
    query: Research about the XZ backdoor
"""

agent.invoke(prompt)



> Entering new AgentExecutor chain...

Invoking: `DuckDuckGoSearchTool` with `{'query': 'Research about the XZ backdoor'}`


[snippet: In February of this year, Tan issued commits for versions 5.6.0 and 5.6.1 of XZ Utils. The updates implemented the backdoor. In the following weeks, Tan or others appealed to developers of Ubuntu ..., title: The XZ Backdoor: Everything You Need to Know - WIRED, link: https://www.wired.com/story/xz-backdoor-everything-you-need-to-know/], [snippet: At a glance, Jia Tan certainly looks East Asian—or is meant to. The time zone of Jia Tan's commits are UTC+8: That's China's time zone, and only an hour off from North Korea's. However ..., title: The Mystery of 'Jia Tan,' the XZ Backdoor Mastermind - WIRED, link: https://www.wired.com/story/jia-tan-xz-backdoor/], [snippet: Executive summary. CVE-2024-3094 is a vulnerability discovered in the open-source library XZ Utils that stems from malicious code that was pushed into the library by one of its maintainers

{'input': '\n    Search for query. Use the any SearchTool.\n    You find a website address, visit the website, extract the content.\n    Save your findings to a text file.\n    query: Research about the XZ backdoor\n',
 'output': 'I have researched and extracted information about the XZ backdoor from the website. The findings have been saved to a text file named "What_is_XZ_backdoor.txt".'}